In [1]:
#import libraries
import cv2
import numpy as np 
import os
import tensorflow as tf
import keras
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from matplotlib import pyplot as plt
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Using TensorFlow backend.


In [4]:
#vars
xs = []
ys = []
n_ys = []

unicode = "2306 2310 2311 2312 2313 2315 2318 2319 2320 2322 2325 2327 2328 2330 2331 2332 2334 2335 2336 2337 2338 2339 2340 2341 2342 2343 2344 2346 2347 2348 2349 2350 2351 2352 2354 2357 2358 2359 2360 2361 2362 2363 2364 2366 2367 2368 2369 2370 2375 2376 2379 2380 2381 2382 2387 2388 2390 2392 2399 2404 2405 2406 2407 2408 2409 2410 2411 2412 2413 2414 2415 2416 2417 2423 2424 2429".split(' ')
unicode = [int(a) for a in unicode] #convert to int
from collections import defaultdict
unicode2idx = defaultdict(int)

idx = 0 
for uni in unicode : 
    unicode2idx[uni] = idx
    idx += 1

idx2unicode = defaultdict(int)
for i in range(idx) :
    idx2unicode[i] = unicode[i]
# print("idx2unicode[1] ", idx2unicode[1])

def get_features(path='./clean') : 
    
    curd = os.getcwd()
    os.chdir(path)
    
    photos =[filename for filename in os.listdir('.') if filename.endswith(".png")]
    
    global xs, ys, n_ys
    n_labels_list = []
    for filename in photos : 
        image = cv2.imread(filename, 0)
        #resize my image to 64 x 64, known to be squares so aspect ratio
        xs.append(image)
        
        #ys
        labels = filename.split('_')[3:]
        labels[-1] = labels[-1].split('.')[0] #remove .extension
        labels = [int(label) for label in labels]
        
        hot_labels = [int(uni in labels) for uni in unicode]
        ys.append(hot_labels)
        #n_ys
        n_labels = len(labels)
        hot_n_labels = [int(i == n_labels) for i in range(1, 4)]
        if sum(hot_n_labels) == 0 : 
            hot_n_labels[2] = 1
        n_ys.append(hot_n_labels)

    #test
    
#     result = ''
#     one_y = ys[0]
#     for idx in range(len(one_y)) : 
#         if one_y[idx] == 1 : 
#             result += str(idx2unicode[idx]) + ' '
#     print(result)
#     print(sum(ys[0]), " == ", n_ys[0])
#     print(photos[0])
    
    #conv lst to ndarray
    xs = np.array(xs)
    ys = np.array(ys)
    n_ys = np.array(n_ys)
    
    #optimise and reshape for tensorflow
    xs = xs.astype('float32')
    xs /= 255
    xs = xs.reshape(xs.shape[0], 64, 64, 1)
    
    print("GET FEATURES X, Y, N_Ys", xs.shape, ys.shape, n_ys.shape)
    os.chdir(curd)
    return 

#os.chdir('/home/ashutosh/Desktop/rohan/prom')
#os.chdir('C:\\Users\\mehul\\Desktop\\weaklearner')
# print(os.chdir('..'))
# print(os.getcwd())
get_features('./clean')

None
/home/deepak/Desktop/weaklearner
('GET FEATURES X, Y, N_Ys', (1729, 64, 64, 1), (1729, 76), (1729, 3))


In [9]:
import os
os.getcwd()

'/home/aw/Desktop/weaklearner'

In [10]:
#helper test
print(len(unicode))

76


In [11]:
#hotcoding check 
for i in range(10) : 
    print(sum(ys[i]), n_ys[i])

2 [0 1 0]
2 [0 1 0]
1 [1 0 0]
2 [0 1 0]
2 [0 1 0]
2 [0 1 0]
1 [1 0 0]
1 [1 0 0]
1 [1 0 0]
1 [1 0 0]


In [12]:
nbatches = 16
nepochs = 1

In [5]:
def train_model(xs, ys) :
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(64, 64, 1)))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides=2))
    
    model.add(Dropout(0.2))
    
    
    model.add(Flatten())
    #dense
    
    model.add(Dense(128))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(ys.shape[1], activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam())   
    model.fit(xs, ys, batch_size = 32, shuffle = True, epochs = 45, verbose = 1, validation_split = 0.10)    
    return model

#test
# print(xs.shape, type(xs))
# print(ys.shape, type(ys))
model = train_model(xs, ys)

Train on 1556 samples, validate on 173 samples
Epoch 1/45
1556/1556 [==============================] - 6s 4ms/step - loss: 0.4221 - val_loss: 0.2498
Epoch 2/45
1556/1556 [==============================] - 4s 3ms/step - loss: 0.1374 - val_loss: 0.1526
Epoch 3/45
1556/1556 [==============================] - 4s 3ms/step - loss: 0.0897 - val_loss: 0.1149
Epoch 4/45
1556/1556 [==============================] - 4s 3ms/step - loss: 0.0713 - val_loss: 0.0999
Epoch 5/45
1556/1556 [==============================] - 4s 3ms/step - loss: 0.0585 - val_loss: 0.0983
Epoch 6/45
1556/1556 [==============================] - 4s 3ms/step - loss: 0.0497 - val_loss: 0.0970
Epoch 7/45
1556/1556 [==============================] - 4s 3ms/step - loss: 0.0416 - val_loss: 0.1002
Epoch 8/45
1556/1556 [==============================] - 4s 3ms/step - loss: 0.0365 - val_loss: 0.0984
Epoch 9/45
1556/1556 [==============================] - 4s 3ms/step - loss: 0.0319 - val_loss: 0.0813
Epoch 10/45
1556/1556 [============

In [6]:
model_name = "model"
n_model_name = "n_model"

In [7]:
import json
def save_model(model, name) : 
    model_json = model.to_json()
    with open(name + ".json", "w") as json_file : 
        json_file.write(model_json)
    model.save_weights(name + ".h5")
    print("saved {} to disk".format(name))
    
#save_model(model, model_name)

In [8]:
save_model(model, model_name)

saved model to disk


In [9]:
def load_model(model_name) : 
    json_file = open(model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(model_name + ".h5")
    print("Loaded {} from disk".format(model_name))
    return loaded_model

In [10]:
model_name = "model"
n_model_name = "n_model"
model = load_model(model_name)

Loaded model from disk


In [15]:
def train_n_model(xs, n_ys) :
    global batch_size, epochs
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1)))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides=2))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))

    
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    
    
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides=2))
    model.add(Dropout(0.2))
    
    
    model.add(Flatten())
    #dense
    
    model.add(Dense(512))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_ys.shape[1], activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics = ['accuracy'])   
    model.fit(xs, n_ys, batch_size = 32, shuffle = True, epochs = 20, verbose = 1, validation_split = 0.10)    
    return model

n_model = train_n_model(xs, n_ys)
save_model(n_model, n_model_name)

Train on 1556 samples, validate on 173 samples
Epoch 1/20
1556/1556 [==============================] - 6s 4ms/step - loss: 0.8389 - acc: 0.7346 - val_loss: 3.2745 - val_acc: 0.4104
Epoch 2/20
1556/1556 [==============================] - 5s 3ms/step - loss: 0.4396 - acc: 0.8490 - val_loss: 4.6396 - val_acc: 0.4104
Epoch 3/20
1556/1556 [==============================] - 5s 3ms/step - loss: 0.2919 - acc: 0.9004 - val_loss: 5.7811 - val_acc: 0.4104
Epoch 4/20
1556/1556 [==============================] - 5s 3ms/step - loss: 0.2062 - acc: 0.9222 - val_loss: 6.0916 - val_acc: 0.4104
Epoch 5/20
1556/1556 [==============================] - 5s 3ms/step - loss: 0.1721 - acc: 0.9389 - val_loss: 6.0076 - val_acc: 0.4104
Epoch 6/20
1556/1556 [==============================] - 5s 3ms/step - loss: 0.1462 - acc: 0.9524 - val_loss: 4.9068 - val_acc: 0.4104
Epoch 7/20
1556/1556 [==============================] - 5s 3ms/step - loss: 0.1453 - acc: 0.9518 - val_loss: 3.8439 - val_acc: 0.4104
Epoch 8/20
1556

In [16]:
model = load_model(model_name)
# model.summary()

Loaded model from disk


In [17]:
n_model = load_model(n_model_name)
# n_model.summary()

Loaded n_model from disk


In [42]:
def process(imagename) : 
    image = cv2.imread(imagename, 0)
    bilateral = cv2.bilateralFilter(image, 0, 15, 23)
    median = cv2.medianBlur(bilateral, 5)
    _, otsu = cv2.threshold(median, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    #padding to 150, 150, [96, x]
    #image.shape[0] = height
    #image.shape[1] = width
    rows, cols = image.shape[0:2]

    #difference to pad up to get square image
    diff = abs(rows - cols)
    one = diff // 2
    other = diff // 2    #integer division
    if diff % 2 != 0 :   #131 - 128 square // squaring k liye
        other += 1

    if rows > cols : 
        pad = cv2.copyMakeBorder(otsu, 0, 0, one, other, cv2.BORDER_CONSTANT, 0)
    else : 
        pad = cv2.copyMakeBorder(otsu, one, other, 0, 0, cv2.BORDER_CONSTANT, 0)

    #bring to uniform size
    size = max(rows, cols)

    if size >= 64 : 
        clean = cv2.resize(pad, (64, 64))
    else : 
        pixel_pad = 64 - size
        one = pixel_pad // 2
        other = pixel_pad // 2
        if pixel_pad % 2 == 1 : 
            other += 1
        clean = cv2.copyMakeBorder(pad, one, other, one, other, cv2.BORDER_CONSTANT, 0)
    return clean

def predict(imagename) : 
    global model, n_model
    global idx2unicode, unicode2idx, unicode
    clean = process(imagename)
    clean = clean.reshape(1, 64, 64, 1)
    clean = clean.astype('float32')
    clean /= 255
    
    n_probs = n_model.predict(clean)[0]
    n_predictions = np.argmax(n_probs) + 1
    
    predictions = model.predict(clean)[0]
    top_n = np.argsort(predictions)[-n_predictions:] #list of toppers (reversed)
    
#     print(result)
    result = []
    #convert idx to unicode 
    for i in top_n : 
        result.append(idx2unicode[i])
    result.reverse()
    
    #test
    print(imagename)
    #print(result)
    return result

In [43]:
# filename = "/home/aw/Desktop/prom/test/page1_15_0_2350_2375.png"
# predict(filename)

In [44]:
import os 

os.chdir('C:\\Users\\mehul\\Desktop\\weaklearner\\clean\\')

photos = [img for img in os.listdir('.') if img.endswith('.png')]
print(len(photos))
sub = photos[0:10]
for e in sub : 
    print(e, predict(e))
#predict('')

1729
page0_0_10_2313.png
page0_0_10_2313.png [2346, 2306]
page0_0_11_2352.png
page0_0_11_2352.png [2360, 2346]
page0_0_13_2340_2367.png
page0_0_13_2340_2367.png [2306, 2346]
page0_0_15_2312.png
page0_0_15_2312.png [2343, 2368]
page0_0_16_2344.png
page0_0_16_2344.png [2366, 2332, 2342]
page0_0_1_2361_2380.png
page0_0_1_2361_2380.png [2332, 2306]
page0_0_2_2312.png
page0_0_2_2312.png [2360]
page0_0_3_2344.png
page0_0_3_2344.png [2342, 2366]
page0_0_5_2346_2368.png
page0_0_5_2346_2368.png [2340, 2354]
page0_0_6_2325_2368.png
page0_0_6_2325_2368.png [2306, 2366]
